In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [ ]:
import tensorflow as tf

print(tf.__version__)

2.0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#m = tf.keras.models.load_model("/content/drive/My Drive/face_mask_model.h5")

ValueError: ignored

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
#print("[INFO] loading images...")
d = '/content/drive/My Drive/train'
imagePaths = list(paths.list_images(d))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
  #print(imagePath)
  # extract the class label from the filename
  label = imagePath.split(os.path.sep)[-2]
  # load the input image (224x224) and preprocess it
  image = load_img(imagePath, target_size=(224, 224))
  image = img_to_array(image)
  image = preprocess_input(image)
  # update the data and labels lists, respectively
  data.append(image)
  labels.append(label)


# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [ ]:
#compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=20)

[INFO] compiling model...
[INFO] training head...
Train for 37 steps, validate on 302 samples
Epoch 1/20
37/37 [==============================] - 21s 569ms/step - loss: 0.7189 - accuracy: 0.6172 - val_loss: 0.4673 - val_accuracy: 0.7743
Epoch 2/20
37/37 [==============================] - 12s 326ms/step - loss: 0.4103 - accuracy: 0.8133 - val_loss: 0.2649 - val_accuracy: 0.9097
Epoch 3/20
37/37 [==============================] - 12s 321ms/step - loss: 0.3137 - accuracy: 0.8696 - val_loss: 0.1946 - val_accuracy: 0.9375
Epoch 4/20
37/37 [==============================] - 12s 325ms/step - loss: 0.2356 - accuracy: 0.9003 - val_loss: 0.1373 - val_accuracy: 0.9618
Epoch 5/20
37/37 [==============================] - 12s 320ms/step - loss: 0.2102 - accuracy: 0.9122 - val_loss: 0.1314 - val_accuracy: 0.9688
Epoch 6/20
37/37 [==============================] - 12s 321ms/step - loss: 0.1864 - accuracy: 0.9250 - val_loss: 0.1134 - val_accuracy: 0.9792
Epoch 7/20
37/37 [==============================

In [ ]:
model.save('/content/drive/My Drive/face_mask_model.h5')